In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import pickle
import h5py
import sys

In [ ]:
database = r""
sklearn_model = r""
jobs = -1

In [ ]:
# open the database
db = h5py.File(database, mode="r")
i = int(db["labels"].shape[0] * 0.75)

In [ ]:
print(f"[INFO] tuning hyperparams......")
params = {"C": [0.0001, 0.001, 0.01, 0.1, 1.0]}
model = GridSearchCV(LogisticRegression(solver="lbfgs",
                                        multi_class="auto"),
                     param_grid=params,
                     cv=3,
                     n_jobs=jobs)

model.fit(db["images"][:i], db["labels"][:i])
print(f"[INFO] best hyperparams: {model.best_params_}......")

In [ ]:
# generate a classification report for the model
print(f"[INFO] evaluating......")
preds = model.predict(db["images"][i:])
print(classification_report(y_true=db["labels"][i:],
                            y_pred=preds,
                            target_names=db["label_names"]))

# calculate accuracy
acc = accuracy_score(db["labels"][i:], preds)
print(f"[INFO] acc score: {acc}......")

In [ ]:
# serialize the model to disk
print(f"[INFO] saving model......")
f = open(sklearn_model, mode="wb")
f.write(pickle.dumps(model.best_estimator_))
f.close()
db.close()